<a href="https://colab.research.google.com/github/ngockhanh5110/nlp-vietnamese-text-summarization/blob/main/notebooks/inferencing_vietnews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# Authenticate
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [3]:
OUTPUT_DIR = '/content/training/'

In [4]:
import random

random.seed(0)

In [5]:
%%capture

!wget 'https://github.com/ThanhChinhBK/vietnews/archive/master.zip'
!unzip 'master.zip'

In [6]:
%%capture

# !wget 'https://github.com/CLC-HCMUS/ViMs-Dataset/raw/master/ViMs.zip'
# !unzip 'ViMs.zip'

# Install the vncorenlp python wrapper
!pip install vncorenlp

# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter) 
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/
!pip install datasets==1.0.2

In [7]:
import glob
import pandas as pd
import concurrent.futures
from datasets import *

## Processing data

In [8]:
# from vncorenlp import VnCoreNLP
# rdrsegmenter = VnCoreNLP("./vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx2g') 

In [9]:
def listPaths(path):
  pathfiles = list()
  for pathfile in glob.glob(path):
    pathfiles.append(pathfile)
  return pathfiles

train_paths = listPaths('/content/vietnews-master/data/train_tokenized/*')
val_paths = listPaths('/content/vietnews-master/data/val_tokenized/*')
test_paths = listPaths('/content/vietnews-master/data/test_tokenized/*')

In [10]:
def read_content(pathfile):
  """
  Input: Path of txt file
  Output: A dictionary has keys 'original' and 'summary'
  """
  with open(pathfile) as f:
    rows  = f.readlines()
    original = ' '.join(''.join(rows[4:]).split('\n'))
    summary = ' '.join(rows[2].split('\n'))
          
  return {'file' : pathfile,
            'original': original, 
            'summary': summary}

In [11]:
read_content(train_paths[0])

{'file': '/content/vietnews-master/data/train_tokenized/033889.txt.seg',
 'original': 'Theo RT , liên_quân do Mỹ dẫn_đầu được cho là đã không_kích các vị_trí của quân_đội Syria tại khu_vực Al_Bukamal thuộc tỉnh Deir ez - Zor , phía Đông_Syria . Thông_tin trên được hãng thông_tấn nhà_nước Syria SANA trích_dẫn nguồn tin quân_sự cho_hay . Theo đó , cuộc không_kích nhằm vào một_số vị_trí của quân_đội Syria tại al - Harra , phía Đông nam khu_vực Al_Bukamal . Theo nguồn_tin này , vụ không_kích dẫn đến tổn_thất về nhân_lực của quân_đội Syria , tuy_nhiên con_số thương_vong cụ_thể chưa được công_bố . Vụ không_kích này có_thể do một_số máy_bay_không_người_lái ( UAV ) của Mỹ thực_hiện , đồng_thời vụ không_kích này được cho là nhằm vào một_số vị_trí của lực_lượng Iraq và quân_đội Chính_phủ Syria tại khu_vực giữa Al_Bukamal và Tanf . Tuy_nhiên , hiện Lầu_Năm_Góc phủ_nhận có liên_quan tới vụ_việc . Đây không phải là vụ không kích đầu_tiên nhằm vào lực_lượng quân_đội Syria và các lực_lượng ủng_hộ Chí

In [12]:
def get_dataframe(pathfiles):
  with concurrent.futures.ProcessPoolExecutor() as executor:
    data = executor.map(read_content, pathfiles)
  
  # Make blank dataframe
  data_df = list()
  for d in data:
    data_df.append(d)
  data_df = pd.DataFrame(data_df)
  data_df.dropna(inplace = True)
  data_df = data_df.sample(frac=1).reset_index(drop=True)

  return data_df

In [13]:
# train_df = get_dataframe(train_paths)

In [14]:
test_df = get_dataframe(test_paths)

## **Warm-starting RoBERTaShared for BBC XSum**

***Note***: This notebook only uses a few training, validation, and test data samples for demonstration purposes. To fine-tune an encoder-decoder model on the full training data, the user should change the training and data preprocessing parameters accordingly as highlighted by the comments.


### **Data Preprocessing**


In [15]:
%%capture
!pip install datasets==1.0.2
!pip install transformers

import datasets
import transformers

In [16]:
from transformers import RobertaTokenizerFast,AutoTokenizer


In [17]:
from sklearn.model_selection import train_test_split

# train_data =  Dataset.from_pandas(train_df)

In [18]:
test_data =  Dataset.from_pandas(test_df)

### **Fine-Tuning Warm-Started Encoder-Decoder Models**

The `Seq2SeqTrainer` that can be found under [examples/seq2seq/seq2seq_trainer.py](https://github.com/huggingface/transformers/blob/master/examples/seq2seq/seq2seq_trainer.py) will be used to fine-tune a warm-started encoder-decoder model.

Let's download the `Seq2SeqTrainer` code and import the module along with `TrainingArguments`.

In [19]:
%%capture
!rm seq2seq_trainer.py
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/seq2seq_trainer.py

!pip install git-python==1.0.3
!pip install sacrebleu==1.4.12
!pip install rouge_score

from seq2seq_trainer import Seq2SeqTrainer
from transformers import TrainingArguments
from dataclasses import dataclass, field
from typing import Optional

We need to add some additional parameters to make `TrainingArguments` compatible with the `Seq2SeqTrainer`. Let's just copy the `dataclass` arguments as defined in [this file](https://github.com/patrickvonplaten/transformers/blob/make_seq2seq_trainer_self_contained/examples/seq2seq/finetune_trainer.py).

Also, we need to define a function to correctly compute the ROUGE score during validation. ROUGE is a much better metric to track during training than only language modeling loss.

In [20]:
import datasets

In [21]:
# load rouge for validation
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

### **Evaluation**

Awesome, we finished training our dummy model. Let's now evaluated the model on the test data. We make use of the dataset's handy `.map()` function to generate a summary of each sample of the test data.

In [24]:
!mkdir 'training'

In [27]:
!gsutil -m cp -r 'gs://kaggle-vbdi-test/training_data/checkpoint-5000/*' '/content/training/' 

Copying gs://kaggle-vbdi-test/training_data/checkpoint-5000/optimizer.pt...
Copying gs://kaggle-vbdi-test/training_data/checkpoint-5000/scheduler.pt...
Copying gs://kaggle-vbdi-test/training_data/checkpoint-5000/trainer_state.json...
Copying gs://kaggle-vbdi-test/training_data/checkpoint-5000/config.json...
Copying gs://kaggle-vbdi-test/training_data/checkpoint-5000/pytorch_model.bin...
Copying gs://kaggle-vbdi-test/training_data/checkpoint-5000/training_args.bin...
/ [6/6 files][  1.8 GiB/  1.8 GiB] 100% Done  48.0 MiB/s ETA 00:00:00           
Operation completed over 6 objects/1.8 GiB.                                      


In [28]:
import datasets
from transformers import RobertaTokenizer, EncoderDecoderModel, AutoTokenizer
from sklearn.model_selection import train_test_split

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

model = EncoderDecoderModel.from_pretrained('/content/training')
model.to("cuda")

# test_data = datasets.load_dataset("xsum", split="test")

batch_size = 32  # change to 64 for full evaluation

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    inputs = tokenizer(batch["original"], padding="max_length", truncation=True, max_length=256, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch

results = test_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["original"])

pred_str = results["pred"]
label_str = results["summary"]

In [29]:
rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge1","rouge2","rougeL"])

In [30]:
for key,value in rouge_output.items():
  print(key)
  print(value.mid)

rouge1
Score(precision=0.5941934140108612, recall=0.519659224472204, fmeasure=0.5429459937735283)
rouge2
Score(precision=0.22187960217910824, recall=0.19439264344591123, fmeasure=0.2028052420247029)
rougeL
Score(precision=0.36592791774158084, recall=0.32054213434134515, fmeasure=0.33452800500084656)


In [70]:
i = 600
print('Prediction: ',pred_str[i])
print('Truth: ',label_str[i])
print('Content: ',test_data[i]['original'])

Prediction:  Trong tỉnh Thừa_Thiên - Huế, ba người chết và mất_tích trong vụ lũ cuốn trôi nhà dân ở huyện Quảng_Điền ( Thừa_Thiên_Huế ).
Truth:  Nước lũ đã rút từ đêm qua , nhưng hôm_nay lại lên do trời mưa to . Toàn tỉnh có hơn 70.000 căn nhà bị ngập , 9 người chết . 
Content:  Sáng 7/11 , lực_lượng chức_năng đã tìm thấy thi_thể ông Phan_Văn_Quốc và con_gái Phan_Thị_Thuý ( 24 tuổi , xã Thuỷ_Thanh , thị_xã Hương_Thuỷ ) . Cha_con ông Quốc bị lũ cuốn mất_tích trên đường đi làm sáng 5/11 . Ngoài hai nạn_nhân này , tỉnh còn 7 người chết do lũ cuốn . Hơn 70.200 căn nhà bị ngập sâu 0,2-0,8 m . 10 km bờ biển sạt_lở , nhiều diện_tích hoa_màu bị ngập , hàng trăm tấn cá nuôi của người_dân bị chết . Nước lũ ở huyện Quảng_Điền còn ngập sâu . Ông Phan_Thanh_Hùng , Ban chỉ_huy phòng_chống thiên_tai và tìm_kiếm cứu nạn tỉnh cho biết , đợt lũ này mực nước lớn nhất trên sông Hương , sông Bồ đều vượt báo_động 3 , trong đó nước sông Bồ xấp_xỉ đỉnh lũ lịch_sử 1999 ( 5,18 m ) . Đêm qua lũ các sông đã giảm 

In [34]:
test_data[i]['file']

'/content/vietnews-master/data/test_tokenized/016385.txt.seg'

In [36]:
from vncorenlp import VnCoreNLP

rdrsegmenter = VnCoreNLP("./vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx2g') 

In [74]:
text = '''
Theo tin từ Ban chỉ huy phòng chống lụt bão tỉnh Quảng Bình, ngày 26-11, mưa lớn đã làm ngập 100 ngôi nhà, chủ yếu ở huyện miền núi Tuyên Hóa, 65ha ngô đông và hàng trăm ha rau màu bị ngập. Hơn 300ha ao hồ nuôi tôm và các loại thủy sản nước ngọt có nguy cơ bị mất trắng. Hiện đã có một người chết do lũ cuốn trôi. Riêng vụ lật đò ngang trên thượng nguồn sông Gianh vào sáng 26-11 đã làm một người mất tích, ba người bị thương nặng.

Tại Quảng Trị, mưa lớn trong các ngày 24, 25 và 26-11 đã khiến nhiều vùng trên địa bàn bị ngập lụt nặng. Đã có hai nạn nhân thiệt mạng trong hai ngày mưa lũ vừa qua. 

Mưa lũ lớn đã làm nhiều đoạn đường Hồ Chí Minh bị sạt lở gây ách tắc giao thông tại các điểm km28, km42, km49 (tính từ cầu Đakrông vào). Chiều 26-11 sạt thêm các đoạn km10, km31, km28. Hiện ở vị trí km31 giao thông đang tắc nghẽn hoàn toàn do đoạn sạt này dài hơn 100m. Tuyến đường du lịch “đường Hồ Chí Minh huyền thoại” cũng bị sạt lở dài hơn 150m từ km28 đi vào theo tuyến xã Tà Long.

Tại Cửa Tùng (huyện Vĩnh Linh), có ba tàu đánh cá của ngư dân thôn An Hòa 3 bị sóng đánh trôi ra biển, nhân dân địa phương đã tổ chức cứu được hai tàu, còn một chiếc không tìm được.

Riêng khu vực đồng bằng huyện Hải Lăng đã chìm sâu trong nước, hệ thống giao thông liên huyện, liên xã bị chia cắt hoàn toàn với hơn 8.000 hộ dân, có nhà bị ngập từ 0,5-1m. Cũng theo báo cáo của huyện này chưa có thiệt hại về người nhưng hiện mọi người đang lo lắng cho số phận của 15 người dân ở xã Hải Lâm đi trồng rừng nay vẫn chưa thấy về...
'''

In [75]:
text = rdrsegmenter.tokenize(text)
text = ' '.join([' '.join(x) for x in text])
text

'Theo tin từ Ban chỉ_huy phòng_chống lụt_bão tỉnh Quảng_Bình , ngày 26-11 , mưa lớn đã làm ngập 100 ngôi nhà , chủ_yếu ở huyện miền núi Tuyên_Hoá , 65ha ngô đông và hàng trăm ha rau_màu bị ngập . Hơn 300ha ao_hồ nuôi tôm và các loại thuỷ_sản nước_ngọt có nguy_cơ bị mất_trắng . Hiện đã có một người chết do lũ cuốn trôi . Riêng vụ lật đò_ngang trên thượng_nguồn sông Gianh vào sáng 26-11 đã làm một người mất_tích , ba người bị_thương nặng . Tại Quảng_Trị , mưa lớn trong các ngày 24 , 25 và 26-11 đã khiến nhiều vùng trên địa_bàn bị ngập_lụt nặng . Đã có hai nạn_nhân thiệt_mạng trong hai ngày mưa_lũ vừa_qua . Mưa_lũ lớn đã làm nhiều đoạn đường Hồ_Chí_Minh bị sạt_lở gây ách_tắc giao_thông tại các điểm km28 , km42 , km49 ( tính từ cầu Đakrông vào ) . Chiều 26-11 sạt thêm các đoạn km10 , km31 , km2 8 . Hiện ở vị_trí km31 giao_thông đang tắc_nghẽn hoàn_toàn do đoạn sạt này dài hơn 100m . Tuyến đường du_lịch “ đường Hồ_Chí_Minh huyền_thoại ” cũng bị sạt_lở dài hơn 150m từ km28 đi vào theo tuyến 

In [76]:
inputs = tokenizer(text, padding="max_length", truncation=True, max_length=256, return_tensors="pt")
input_ids = inputs.input_ids.to("cuda")
attention_mask = inputs.attention_mask.to("cuda")

outputs = model.generate(input_ids, attention_mask=attention_mask)

# all special tokens including will be removed
output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
output_str

['Mưa mưa lớn kéo_dài khiến nhiều tuyến đường ở Quảng_Bình bị ngập sâu, hàng chục nhà dân bị tốc mái, hàng trăm ngôi nhà bị sập.']